<h1>Mobility Planner</h1>

This notebook was produced in the context of the Applied Data Science Capstone Project "The Battle of Neighborhoods", that is a part of the [IBM Data Science Professional Certificate offered on Coursera](https://www.coursera.org/specializations/ibm-data-science).

Below here, I will try building a tool that could be used by public decision makers for planning the strengthening of mobility-related services and infrastructures. 

The tool highlights those areas where mobility services appear not to be adequate given the attractiveness of the area.

See https://github.com/mircosoderi/mobilityplanner to learn more.

<h3>How to use this notebook</h3>

The notebook is partitioned into a sequence of steps.

At each step, you will find one of these:

<p><span style="font-size:larger;">&#128583;</span> means <i>Take action!</i>, it is where you have to make your hands dirty</p>
<p><span style="font-size:larger;">&#128582;</span> means <i>Enjoy!</i>, it is where you just have to run cells and relax</p>

<h3> &#128583; Step 1 - Configuration</h3>

In the code cell below here, indicate the city for which you wish to perform the analysis, and other parameters. 

In [1]:
# For which city do you wish to perform the analysis?
city = "Montelupo Fiorentino"

# How about the minimum size of each squared partition of the territory subject to analysis?
min_side_length = 500

# How about the impact of private and public transport facilities and infrastructures over the quantification of quality of mobility?
pvt_tpt_wgt = 1
pbl_tpt_wgt = 2

# How about the impact of Open Street Map nodes and FourSquare likes over the quantification of attractiveness?
osm_nodes_wgt = 1
fs_likes_wgt = 2

<h3> &#128582; Step 2 - Bounding box of the city</h3>

Just execute the cell below here to query the Open Street Map for the city that you have specified above, and get back its bounding box.

In [2]:
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import pandas as pd
geolocator = Nominatim(user_agent="ny_explorer")
bbox = pd.DataFrame(geolocator.geocode(city,exactly_one=True).raw)['boundingbox']
lat_min = float(bbox[0])
lon_min = float(bbox[2])
lat_max = float(bbox[1])
lon_max = float(bbox[3])
print(lat_min,lon_min,lat_max,lon_max)

43.6933031 10.9760734 43.7675612 11.06845


<h3> &#128582; Step 3 - Bounding box partitioning</h3>

The bounding box is partitioned in squares whose side is the 1% of the minimum between the height and the width of the bounding box obtained at Step 2, or _min_side_length_ if less.

Run cells here below to initialize the table of partitions.

In [3]:
class Partition:
    lat_min = 0
    lon_min = 0
    lat_max = 0
    lon_max = 0
    osm_df = 0
    osm_nodes = 0    
    osm_pbl_tpt = 0
    osm_highways = 0
    osm_parkings = 0
    osm_rentals = 0
    fs_df = 0
    fs_likes = 0
    attractiveness_val = 0
    attractiveness_cat = 0
    mobility_val = 0
    mobility_cat = 0
    gap = 0
print("Class definition: OK")

Class definition: OK


In [4]:
from math import pi, cos, ceil
# See https://amsi.org.au/ESA_Senior_Years/SeniorTopic8/8b/8b_2content_5.html for:
bbox_h_m = abs ( 6400 * pi / 180 * ( lat_max - lat_min ) ) * 1000
# See http://spmmathematics.blog.onlinetuition.com.my/2016/05/942-distance-between-two-points-along.html for:
bbox_w_m = ( lon_max - lon_min ) * 60 * cos( ( lat_max + lat_min ) / 2 ) * 1.852 * 1000
# side length = 1% of min(bboxHeight,bboxWidth) or 100 m if less
side_l_m = max(min_side_length,min(bbox_h_m,bbox_w_m)/100)
# inverse formulas to calculate delta lat/lon, given the side length
delta_lat = side_l_m / 1000 / 6400 / pi * 180
delta_lon = side_l_m / 60 / cos( ( lat_max + lat_min ) / 2 ) / 1.852 / 1000
# shaping table of territory partitions
partitions_w = ceil( ( lon_max - lon_min ) / delta_lon )
partitions_h = ceil( ( lat_max - lat_min ) / delta_lat )
partitions_tbl = [[0 for x in range(partitions_w)] for y in range(partitions_h)] 
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x] = Partition();
        partitions_tbl[y][x].lat_max = lat_max - y*delta_lat
        partitions_tbl[y][x].lon_min = lon_min + x*delta_lon
        partitions_tbl[y][x].lat_min = partitions_tbl[y][x].lat_max - delta_lat
        partitions_tbl[y][x].lon_max = partitions_tbl[y][x].lon_min + delta_lon
print("Partitions initialization: OK")

Partitions initialization: OK


<h3> &#128582; Step 4 - Open Street Map querying</h3>

Run the cells here below to start filling partitions with data that can be gathered from the Open Street Map

In [5]:
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x].osm_df = pd.read_csv("https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node("+str(partitions_tbl[y][x].lat_min)+","+str(partitions_tbl[y][x].lon_min)+","+str(partitions_tbl[y][x].lat_max)+","+str(partitions_tbl[y][x].lon_max)+");%20%3C;%20);%20out%20meta;",sep='\t')
        print("https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node("+str(partitions_tbl[y][x].lat_min)+","+str(partitions_tbl[y][x].lon_min)+","+str(partitions_tbl[y][x].lat_max)+","+str(partitions_tbl[y][x].lon_max)+");%20%3C;%20);%20out%20meta;")

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.763084967225545,10.9760734,43.7675612,10.980719631760879);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.763084967225545,10.980719631760879,43.7675612,10.985365863521757);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.763084967225545,10.985365863521757,43.7675612,10.990012095282635);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.763084967225545,10.990012095282633,43.7675612,10.994658327043512);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.75860873445109,11.055059339934923,43.763084967225545,11.059705571695801);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.75860873445109,11.059705571695801,43.763084967225545,11.06435180345668);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.75860873445109,11.06435180345668,43.763084967225545,11.068998035217557);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.75413250167663,10.9760734,43.75860873445109,10.980719631760879);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cy

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.749656268902164,11.04112064465229,43.75413250167662,11.045766876413168);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.749656268902164,11.045766876413168,43.75413250167662,11.050413108174046);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.749656268902164,11.050413108174046,43.75413250167662,11.055059339934925);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.749656268902164,11.055059339934923,43.75413250167662,11.059705571695801);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.74070380335325,11.027181949369657,43.74518003612771,11.031828181130535);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.74070380335325,11.031828181130534,43.74518003612771,11.036474412891412);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.74070380335325,11.036474412891412,43.74518003612771,11.04112064465229);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.74070380335325,11.04112064465229,43.74518003612771,11.045766876413168);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highw

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.731751337804326,11.013243254087023,43.736227570578784,11.0178894858479);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.731751337804326,11.0178894858479,43.736227570578784,11.022535717608779);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.731751337804326,11.022535717608779,43.736227570578784,11.027181949369657);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.731751337804326,11.027181949369657,43.736227570578784,11.031828181130535);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.72279887225541,10.99930455880439,43.72727510502987,11.003950790565268);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.72279887225541,11.003950790565268,43.72727510502987,11.008597022326146);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.72279887225541,11.008597022326144,43.72727510502987,11.013243254087023);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.72279887225541,11.013243254087023,43.72727510502987,11.0178894858479);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highwa

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.71384640670649,10.985365863521757,43.718322639480945,10.990012095282635);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.71384640670649,10.990012095282633,43.718322639480945,10.994658327043512);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.71384640670649,10.994658327043512,43.718322639480945,10.99930455880439);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.71384640670649,10.99930455880439,43.718322639480945,11.003950790565268);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,h

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.70937017393203,11.06435180345668,43.71384640670649,11.068998035217557);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.70489394115757,10.9760734,43.70937017393203,10.980719631760879);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.70489394115757,10.980719631760879,43.70937017393203,10.985365863521757);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.70489394115757,10.985365863521757,43.70937017393203,10.990012095282635);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycl

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.700417708383114,11.050413108174046,43.70489394115757,11.055059339934925);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.700417708383114,11.055059339934923,43.70489394115757,11.059705571695801);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.700417708383114,11.059705571695801,43.70489394115757,11.06435180345668);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.700417708383114,11.06435180345668,43.70489394115757,11.068998035217557);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,h

https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.69146524283419,11.036474412891412,43.69594147560865,11.04112064465229);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.69146524283419,11.04112064465229,43.69594147560865,11.045766876413168);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.69146524283419,11.045766876413168,43.69594147560865,11.050413108174046);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highway,cycleway,amenity)];(%20node(43.69146524283419,11.050413108174046,43.69594147560865,11.055059339934925);%20%3C;%20);%20out%20meta;
https://overpass.kumi.systems/api/interpreter?data=[out:csv(::id,::type,public_transport,highw

<h3> &#128582; Step 5 - Open Street Map exploitation</h3>

Run the cell here below to quantify the quality of mobility for each partition, and start quantifying attractiveness, based on data gathered from the Open Street Map at the previous step.

In [6]:
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x].osm_nodes = len(partitions_tbl[y][x].osm_df[partitions_tbl[y][x].osm_df["@type"] == "node"])
        partitions_tbl[y][x].osm_pbl_tpt = partitions_tbl[y][x].osm_df["public_transport"].count()
        partitions_tbl[y][x].osm_highways = partitions_tbl[y][x].osm_df["highway"].count()
        partitions_tbl[y][x].osm_parkings = len(partitions_tbl[y][x].osm_df[partitions_tbl[y][x].osm_df["amenity"] == "parking"])
        partitions_tbl[y][x].osm_rentals = len(partitions_tbl[y][x].osm_df[partitions_tbl[y][x].osm_df["amenity"] == "bicycle_rental"])
        partitions_tbl[y][x].mobility_val = pvt_tpt_wgt * (partitions_tbl[y][x].osm_highways+partitions_tbl[y][x].osm_parkings+partitions_tbl[y][x].osm_rentals) + \
                                        pbl_tpt_wgt * partitions_tbl[y][x].osm_pbl_tpt        
        partitions_tbl[y][x].attractiveness_val = osm_nodes_wgt*partitions_tbl[y][x].osm_nodes
print("Open Street Map exploitation: OK")

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Open Street Map exploitation: OK


<h3> &#128582; Step 6 - Clustering by mobility</h3>

Run the cell here below to cluster partitions of territory on the basis of the quantification of the quality of mobility in each of them.

In [7]:
from sklearn.cluster import KMeans 
k_means_mobility = KMeans(init = "k-means++", n_clusters = 3, n_init = 12)
pts_mobility = []
for y in range(partitions_h):
    for x in range(partitions_w):
        pts_mobility.append([partitions_tbl[y][x].mobility_val])
k_means_mobility.fit(pts_mobility)
result_mobility_clustering = k_means_mobility.labels_
centers_mobility_clustering = k_means_mobility.cluster_centers_
i = 0
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x].mobility_cat = result_mobility_clustering[i]
        print("Partition ("+str(y)+", "+str(x)+") is in cluster "+str(partitions_tbl[y][x].mobility_cat))
        i += 1

Partition (0, 0) is in cluster 1
Partition (0, 1) is in cluster 1
Partition (0, 2) is in cluster 1
Partition (0, 3) is in cluster 1
Partition (0, 4) is in cluster 1
Partition (0, 5) is in cluster 1
Partition (0, 6) is in cluster 1
Partition (0, 7) is in cluster 1
Partition (0, 8) is in cluster 1
Partition (0, 9) is in cluster 1
Partition (0, 10) is in cluster 1
Partition (0, 11) is in cluster 1
Partition (0, 12) is in cluster 1
Partition (0, 13) is in cluster 1
Partition (0, 14) is in cluster 1
Partition (0, 15) is in cluster 1
Partition (0, 16) is in cluster 1
Partition (0, 17) is in cluster 0
Partition (0, 18) is in cluster 1
Partition (0, 19) is in cluster 1
Partition (1, 0) is in cluster 1
Partition (1, 1) is in cluster 1
Partition (1, 2) is in cluster 1
Partition (1, 3) is in cluster 1
Partition (1, 4) is in cluster 1
Partition (1, 5) is in cluster 1
Partition (1, 6) is in cluster 1
Partition (1, 7) is in cluster 1
Partition (1, 8) is in cluster 1
Partition (1, 9) is in cluster 1


<h3> &#128583; Step 7 - Map cluster labels to mobility categories</h3>

You are not granted that lowest values are grouped in cluster 0, medium in cluster 1, and highest in cluster 2. 

Check for _centers_mobility_clustering_ and manually map cluster labels to mobility levels if necessary. 

In [8]:
centers_mobility_clustering

array([[ 32.77777778],
       [  4.78145695],
       [138.        ]])

In [9]:
# In my case, centers_mobility_clustering was
# array([[ 32.77777778],
#        [  4.78145695],
#        [138.        ]])
# So, my mapping is:
for y in range(partitions_h):
    for x in range(partitions_w):
        if partitions_tbl[y][x].mobility_cat == 0:
            mobility = 1
        if partitions_tbl[y][x].mobility_cat == 1:
            mobility = 0
        if partitions_tbl[y][x].mobility_cat == 2:
            mobility = 2
        partitions_tbl[y][x].mobility_cat = mobility
            

<h3> &#128582; Step 8 - Map of Mobility</h3>

Run the cell here below to display the map of the quality of mobility in the territory of your interest.

__Remark__: the map displays _relative_ quantifications of mobility, i.e. you find marked in red those areas that are less served than others _within the boundaries of the considered territory_.

In [11]:
# Install folium if it is not already installed in your runtime environment. I use it for composing maps.
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [12]:
import folium 
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

map_mobility = folium.Map(location=[(lat_min+lat_max)/2, (lon_min+lon_max)/2], zoom_start=13)
rainbow = [colors.rgb2hex(colors.BASE_COLORS["r"]), colors.rgb2hex(colors.BASE_COLORS["y"]), colors.rgb2hex(colors.BASE_COLORS["g"])]
markers_colors = []
for y in range(partitions_h):
    for x in range(partitions_w):
        label = folium.Popup("Mobility here is "+str(partitions_tbl[y][x].mobility_cat)+" in a 0 - 2 scale", parse_html=True)
        lat = (partitions_tbl[y][x].lat_min+partitions_tbl[y][x].lat_max)/2
        lon = (partitions_tbl[y][x].lon_min+partitions_tbl[y][x].lon_max)/2        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[partitions_tbl[y][x].mobility_cat],
            fill=True,
            fill_color=rainbow[partitions_tbl[y][x].mobility_cat],
            fill_opacity=0.7).add_to(map_mobility)       
map_mobility

<h3> &#128582; Step 9 - FourSquare querying and exploitation</h3>

Run the cells here below to query FourSquare for the most relevant venues in the territory of your interest, and for likes got by such venues, that contribute to the quantification of the attractiveness of each partition.

In [13]:
import requests
fs_client_id = 'LH053BNQIPGKBI0WU0EP5UZ124HLZXVX4Z55O1HOYNB5WB2L'
fs_client_secret = 'AW0NG4SS111TVCBGHXFXUTIBFF1OG3HEHJRCUJSST32NA2DH'
fs_version = '20180604'
fs_limit = 50
fs_venues = requests.get('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(fs_client_id, fs_client_secret, (lat_min+lat_max)/2, (lon_min+lon_max)/2, fs_version, fs_limit)).json()
fs_venues

{'meta': {'code': 200, 'requestId': '5f061d53a6e8ba1788a9efbc'},
 'response': {'venues': [{'id': '4e36e49cb61cddd1cd4ca3b5',
    'name': 'Bar Centofiori',
    'location': {'lat': 43.729619653955375,
     'lng': 11.020794727396865,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.729619653955375,
       'lng': 11.020794727396865}],
     'distance': 148,
     'postalCode': '50056',
     'cc': 'IT',
     'city': 'Montelupo Fiorentino',
     'state': 'Toscana',
     'country': 'Italia',
     'formattedAddress': ['50056 Montelupo Fiorentino Toscana', 'Italia']},
    'categories': [],
    'referralId': 'v-1594236194',
    'hasPerk': False},
   {'id': '50086ce6e4b018497f90a8ca',
    'name': 'Fontanello Montelupo',
    'location': {'lat': 43.73023601333479,
     'lng': 11.019602005782795,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.73023601333479,
       'lng': 11.019602005782795}],
     'distance': 215,
     'cc': 'IT',
     'country': 'Italia',
     'formattedA

In [14]:
for y in range(partitions_h):
    for x in range(partitions_w):
        for venue in fs_venues["response"]["venues"]:
            if venue["location"]["lat"] >= partitions_tbl[y][x].lat_min and venue["location"]["lat"] <= partitions_tbl[y][x].lat_max and venue["location"]["lng"] >= partitions_tbl[y][x].lon_min and venue["location"]["lng"] <= partitions_tbl[y][x].lon_max:
                fs_likes = requests.get('https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(venue["id"],fs_client_id,fs_client_secret,fs_version)).json()
                partitions_tbl[y][x].attractiveness_val = partitions_tbl[y][x].attractiveness_val + fs_likes_wgt*fs_likes["response"]["likes"]["count"]
                print(venue["name"]+" in partition ("+str(y)+", "+str(x)+") was found to have "+str(fs_likes["response"]["likes"]["count"])+" likes")
                

Mercato Settimanale Montelupo f.no in partition (7, 8) was found to have 0 likes
Ristorante Pizzeria Il Pianeta in partition (7, 9) was found to have 7 likes
Vanna Cart & Copy in partition (7, 9) was found to have 0 likes
Segni Strani Tatuaggi in partition (7, 9) was found to have 0 likes
La Volpe e l'Uva in partition (7, 9) was found to have 1 likes
Osteria del sole in partition (7, 9) was found to have 2 likes
Montelupo Fiorentino centro storico in partition (7, 9) was found to have 10 likes
Linea Fontani in partition (7, 9) was found to have 0 likes
Osteria Livornese in partition (7, 9) was found to have 13 likes
Cinema Mignon in partition (7, 9) was found to have 5 likes
Cappellini's in partition (7, 9) was found to have 0 likes
Capofrio in partition (7, 9) was found to have 1 likes
cimitero sammiatello in partition (7, 9) was found to have 0 likes
Bar Pizzeria Seta in partition (7, 9) was found to have 3 likes
Malagigi in partition (7, 9) was found to have 5 likes
Stazione Montelu

<h3> &#128582; Step 10 - Clustering by attractiveness</h3>

Run the cell here below to cluster partitions of territory on the basis of the quantification of the attractiveness of each of them.

In [15]:
from sklearn.cluster import KMeans 
k_means_attractiveness = KMeans(init = "k-means++", n_clusters = 3, n_init = 12)
pts_attractiveness = []
for y in range(partitions_h):
    for x in range(partitions_w):
        pts_attractiveness.append([partitions_tbl[y][x].attractiveness_val])
k_means_attractiveness.fit(pts_attractiveness)
result_attractiveness_clustering = k_means_attractiveness.labels_
centers_attractiveness_clustering = k_means_attractiveness.cluster_centers_
i = 0
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x].attractiveness_cat = result_attractiveness_clustering[i]
        print("Partition ("+str(y)+", "+str(x)+") is in cluster "+str(partitions_tbl[y][x].attractiveness_cat))
        i += 1

Partition (0, 0) is in cluster 0
Partition (0, 1) is in cluster 0
Partition (0, 2) is in cluster 0
Partition (0, 3) is in cluster 0
Partition (0, 4) is in cluster 0
Partition (0, 5) is in cluster 0
Partition (0, 6) is in cluster 0
Partition (0, 7) is in cluster 0
Partition (0, 8) is in cluster 0
Partition (0, 9) is in cluster 0
Partition (0, 10) is in cluster 0
Partition (0, 11) is in cluster 0
Partition (0, 12) is in cluster 0
Partition (0, 13) is in cluster 2
Partition (0, 14) is in cluster 0
Partition (0, 15) is in cluster 0
Partition (0, 16) is in cluster 0
Partition (0, 17) is in cluster 2
Partition (0, 18) is in cluster 0
Partition (0, 19) is in cluster 0
Partition (1, 0) is in cluster 0
Partition (1, 1) is in cluster 0
Partition (1, 2) is in cluster 0
Partition (1, 3) is in cluster 0
Partition (1, 4) is in cluster 0
Partition (1, 5) is in cluster 0
Partition (1, 6) is in cluster 0
Partition (1, 7) is in cluster 0
Partition (1, 8) is in cluster 0
Partition (1, 9) is in cluster 0


<h3> &#128583; Step 11 - Map cluster labels to attractiveness levels</h3>

You are not granted that lowest values are in cluster 0, medium in cluster 1, and highest in cluster 2. 

Check for _centers_attractiveness_clustering_ and manually map cluster labels to attractiveness levels if necessary.

In [16]:
centers_attractiveness_clustering

array([[112.47],
       [983.  ],
       [478.5 ]])

In [17]:
# In my case, centers_attractiveness_clustering was
# array([[112.47],
#        [983.  ],
#        [478.5 ]])
# So, my mapping is:
for y in range(partitions_h):
    for x in range(partitions_w):
        if partitions_tbl[y][x].attractiveness_cat == 0:
            attractiveness = 0
        if partitions_tbl[y][x].attractiveness_cat == 1:
            attractiveness = 2
        if partitions_tbl[y][x].attractiveness_cat == 2:
            attractiveness = 1
        partitions_tbl[y][x].attractiveness_cat = attractiveness

<h3> &#128582; Step 12 - Map of Attractiveness</h3>

Run the cell here below to display the map of the attractiveness in the territory of your interest.

__Remark__: the map displays _relative_ quantifications of attractiveness, i.e. you find marked in red those areas that are less attractive than others _within the boundaries of the considered territory_.

In [18]:
map_attractiveness = folium.Map(location=[(lat_min+lat_max)/2, (lon_min+lon_max)/2], zoom_start=13)
rainbow = [colors.rgb2hex(colors.BASE_COLORS["r"]), colors.rgb2hex(colors.BASE_COLORS["y"]), colors.rgb2hex(colors.BASE_COLORS["g"])]
markers_colors = []
for y in range(partitions_h):
    for x in range(partitions_w):
        label = folium.Popup("Attractiveness here is "+str(partitions_tbl[y][x].attractiveness_cat)+" in a 0 - 2 scale", parse_html=True)
        lat = (partitions_tbl[y][x].lat_min+partitions_tbl[y][x].lat_max)/2
        lon = (partitions_tbl[y][x].lon_min+partitions_tbl[y][x].lon_max)/2        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[partitions_tbl[y][x].attractiveness_cat],
            fill=True,
            fill_color=rainbow[partitions_tbl[y][x].attractiveness_cat],
            fill_opacity=0.7).add_to(map_attractiveness)       
map_attractiveness

<h3> &#128582; Step 13 - Gap computation</h3>

Run the cell here below to compute the gap among attractiveness and level of mobility services and infrastructures, for each partition of the territory of interest.

In [19]:
for y in range(partitions_h):
    for x in range(partitions_w):
        partitions_tbl[y][x].gap = partitions_tbl[y][x].mobility_cat - partitions_tbl[y][x].attractiveness_cat + 2

<h3> &#128582; Step 14 - Gap visualization</h3>

Run the cell here below to display a map where areas that would need investments in mobility services and infrastructures are highlighted in red, areas that are very well served are marked in green, intermediate areas are marked with intermediate colors.

In [60]:
map_gap = folium.Map(location=[(lat_min+lat_max)/2, (lon_min+lon_max)/2], zoom_start=13)
rainbow = [colors.rgb2hex(colors.BASE_COLORS["r"]), colors.rgb2hex(colors.TABLEAU_COLORS["tab:orange"]), colors.rgb2hex(colors.BASE_COLORS["y"]), colors.rgb2hex(colors.TABLEAU_COLORS["tab:green"]), colors.rgb2hex(colors.BASE_COLORS["g"])]
markers_colors = []
for y in range(partitions_h):
    for x in range(partitions_w):
        label = folium.Popup("Gap here is "+str(partitions_tbl[y][x].gap)+" in a 0 - 4 scale", parse_html=True)
        lat = (partitions_tbl[y][x].lat_min+partitions_tbl[y][x].lat_max)/2
        lon = (partitions_tbl[y][x].lon_min+partitions_tbl[y][x].lon_max)/2        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[partitions_tbl[y][x].gap],
            fill=True,
            fill_color=rainbow[partitions_tbl[y][x].gap],
            fill_opacity=0.7).add_to(map_gap)       
map_gap